# Overview
This is version 1.0 of the model. It is my better effort to create a model based on the specifications of evolving hockey in the following write up.
https://evolving-hockey.com/blog/a-new-expected-goals-model-for-predicting-goals-in-the-nhl/

In [13]:
#Import libraries
#used to import data
import pandas as pd
#XGB is the model that is used
import xgboost as xgb
from xgboost import cv
#Evaluates the model
from sklearn.metrics import log_loss

import time #just used to see how long the model takes to run

In [5]:
''' In their write up they say that for even strength, they used data from 10-11 to 16-17
    Then they tested it on 17-18 data that the model had not seen. 
    I'm using this strategy to see if my model performs roughly the same as theirs
'''
df_11 = pd.read_csv('data/pbp/ready_to_model/10_11_pbp_cleaned.csv')
df_12 = pd.read_csv('data/pbp/ready_to_model/11_12_pbp_cleaned.csv')
df_13 = pd.read_csv('data/pbp/ready_to_model/12_13_pbp_cleaned.csv')
df_14 = pd.read_csv('data/pbp/ready_to_model/13_14_pbp_cleaned.csv')
df_15 = pd.read_csv('data/pbp/ready_to_model/14_15_pbp_cleaned.csv')
df_16 = pd.read_csv('data/pbp/ready_to_model/15_16_pbp_cleaned.csv')
df_17 = pd.read_csv('data/pbp/ready_to_model/16_17_pbp_cleaned.csv')
df_18 = pd.read_csv('data/pbp/ready_to_model/17_18_pbp_cleaned.csv')
df = pd.concat([df_11,df_12,df_13,df_14,df_15,df_16,df_17])

In [7]:
def get_X_y(df):
    ''' This function takes in a dataframe and splits it into the X and y variables
    '''
    X = df.drop('is_goal', axis=1)
    y = df.is_goal
    
    return X,y

In [12]:
X_train, y_train = get_X_y(df)
X_test, y_test = get_X_y(df_18)

xgb_reg = xgb.XGBRegressor(objective='binary:logistic',
                           gamme = .12, 
                           eval_metric = 'logloss',
                           #eval_metric = 'auc', 
                           eta = .068,
                           subsample = .78,
                           colsample_bytree = .76,
                           min_child_weight = 9,
                           max_delta_step = 5,
                           nthread = 4)

start = time.time()
xgb_reg.fit(X_train, y_train)
print(start-time.time())

y_pred = xgb_reg.predict(X_test)
print(log_loss(y_test, y_pred))

[19:05:38] WARNING: ..\src\learner.cc:541: 
Parameters: { gamme } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


-21.113216400146484
0.19502098263440848


In [ ]:
#With k-fold validation

In [18]:
data_dmatrix = xgb.DMatrix(data=X_train,label=y_train)
params = {'objective':'binary:logistic','eval_metric':'logloss','eta':.068,
          'subsample':.78,'colsample_bytree':.76,'min_child_weight':9,
          'max_delta_step':5,'nthread':4}
xgb_cv = cv(dtrain=data_dmatrix, params=params, nfold=5, num_boost_round=20, metrics = 'logloss',seed=42)

In [19]:
y_pred = xgb_cv.predict(X_test)

AttributeError: 'DataFrame' object has no attribute 'predict'

In [20]:
print(xgb_cv)

    train-logloss-mean  train-logloss-std  test-logloss-mean  test-logloss-std
0             0.640512           0.000029           0.640619          0.000045
1             0.594916           0.000094           0.594883          0.000140
2             0.554483           0.000070           0.554619          0.000166
3             0.518974           0.000071           0.519033          0.000177
4             0.487394           0.000136           0.487482          0.000184
5             0.459220           0.000138           0.459335          0.000212
6             0.433962           0.000132           0.434081          0.000252
7             0.411311           0.000155           0.411454          0.000284
8             0.390955           0.000167           0.391100          0.000308
9             0.372635           0.000283           0.372804          0.000346
10            0.356026           0.000276           0.356209          0.000361
11            0.341027           0.000284           